# TITANIC DATASET ANALYSIS

Import every necessary library.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Construct the database and dropping unnecessary columns.

In [ ]:
data = pd.read_csv('titanic.csv');
print(data.head())